In [2]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [3]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [4]:

from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [5]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

### Load training

In [6]:
x_tr_NF = pd.read_csv('data/x_train.csv')
y_tr_N = pd.read_csv('data/y_train.csv').values.squeeze()
print(f"Loaded training data: x_tr shape {x_tr_NF.shape}, y_tr shape {y_tr_N.shape}")

Loaded training data: x_tr shape (6346, 7729), y_tr shape (6346,)


### Load validation set

In [7]:
x_va_NF = pd.read_csv('data/x_valid.csv').values
y_va_N = pd.read_csv('data/y_valid.csv').values.squeeze()
print(f"Loaded validation data: x_va shape {x_va_NF.shape}, y_va shape {y_va_N.shape}")

Loaded validation data: x_va shape (792, 7729), y_va shape (792,)


### Load test set 

In [8]:
x_te_NF = pd.read_csv('data/x_test.csv').values
y_te_N = pd.read_csv('data/y_test.csv').values.squeeze()
print(f"Loaded test data: x_te shape {x_te_NF.shape}, y_te shape {y_te_N.shape}")

Loaded test data: x_te shape (793, 7729), y_te shape (793,)


### Load vocabulary as a list of strings

In [9]:
vocab_list = x_tr_NF.columns
print(f"First 10 words: {vocab_list[:5].to_list()}")

x_tr_NF = x_tr_NF.values

First 10 words: ['good', 'great', 'time', 'book', "don't"]


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [24]:
from sklearn.model_selection import PredefinedSplit

x_tr_va_NF = np.vstack([x_tr_NF, x_va_NF])
y_tr_va_N = np.hstack([y_tr_N, y_va_N])

# Get the size of the original training set
N_tr = x_tr_NF.shape[0]
N_va = x_va_NF.shape[0]

# -1 for training samples, 0 for validation
test_fold = np.concatenate([
    -1 * np.ones(N_tr, dtype=int),
     0 * np.ones(N_va, dtype=int)
])

my_split = PredefinedSplit(test_fold)

print(my_split)
print(test_fold.shape)

PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0]))
(7138,)


# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [ ]:

simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101
)

print("small_tree classifier object created.")

small_tree classifier object created.


### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [ ]:
simple_tree.fit(x_tr_NF, y_tr_N)


DecisionTreeClassifier(max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [ ]:
pretty_print_sklearn_tree(simple_tree, vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

The Question: The tree is asking a math question: Is X['great'] <= 0.50?

The Feature: Your features are binary:

0 if the word is absent.

1 if the word is present.

Yes there is an internal class like that

      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)

Yes, it is very common and makes perfect sense for a node to split into two children that predict the same class.

The reason is that the decision tree's goal at each step is not to change the final prediction, but to create two new groups that are as pure as possible.

The algorithm uses a metric like Gini Impurity to decide the best split. A "pure" node is one where all the samples belong to a single class (e.g., 100% positive). The tree will make any split that reduces the overall impurity, even if the majority class in both new groups remains the same.


Even though both groups  predict Negative, the algorithm considers this a success. It has successfully isolated a very confidently negative group from a less certain one.
The best choice at each step is to create two cleaner, but still negative-predicting, groups.

## 1B : Find best Decision Tree with grid search

In [36]:
from sklearn.model_selection import GridSearchCV


# Define the base estimator
base_tree = sklearn.tree.DecisionTreeClassifier(criterion='gini', random_state=101)

param_grid_dt = {
    'max_depth': [2, 8, 32, 128],
    'min_samples_leaf': [1, 3, 9],
    'random_state': [101]
}

searcher_dt = GridSearchCV(
    estimator=base_tree,
    param_grid=param_grid_dt,
    scoring='balanced_accuracy',  
    cv=my_split,             
    return_train_score=True,
    refit=False,                
)

print("GridSearchCV object for Decision Tree created.")

GridSearchCV object for Decision Tree created.


In [38]:

searcher_dt.fit(x_tr_va_NF, y_tr_va_N)

print("Best parameters found:")
print(searcher_dt.best_params_)

print("\nTop 5 results:")
results_dt_df = pd.DataFrame(searcher_dt.cv_results_)
rel_cols = [c for c in results_dt_df.columns if 'param_' in c or 'mean_test_score' in c]
print(results_dt_df[rel_cols].sort_values('mean_test_score', ascending=False).head())

Best parameters found:
{'max_depth': 32, 'min_samples_leaf': 3, 'random_state': 101}

Top 5 results:
   param_max_depth param_min_samples_leaf param_random_state  mean_test_score
7               32                      3                101         0.732380
9              128                      1                101         0.730313
6               32                      1                101         0.727378
11             128                      9                101         0.723193
10             128                      3                101         0.716495


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell



In [41]:
best_tree = base_tree .set_params(**searcher_dt.best_params_)
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=32, min_samples_leaf=3, random_state=101)

### Interpret the best decision tree

In [40]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 817 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    6 are leaves
- depth   5 has   20 nodes, of which    5 are leaves
- depth   6 has   30 nodes, of which   14 are leaves
- depth   7 has   32 nodes, of which   18 are leaves
- depth   8 has   28 nodes, of which   12 are leaves
- depth   9 has   32 nodes, of which   11 are leaves
- depth  10 has   42 nodes, of which   21 are leaves
- depth  11 has   42 nodes, of which   23 are leaves
- depth  12 has   38 nodes, of which   17 are leaves
- depth  13 has   42 nodes, of which   21 are leaves
- depth  14 has   42 nodes, of which   25 are leaves
- depth  15 has   34 nodes, of which   18 are leaves
- depth  16 has   32 nodes, of which   14 are leaves
- depth  17 has   36 nodes, of which   23 are leaves
- dep

Best Max_depth and min_sample_leaf  is 32 , 3 repectively.

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [42]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [43]:
simple_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=3, random_state=101)

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [46]:
importances = simple_forest.feature_importances_
feature_imp_df = pd.DataFrame({
    'feature': vocab_list,
    'importance': importances
})

# Sort by importance, highest first
feature_imp_df = feature_imp_df.sort_values(by='importance', ascending=False)

# --- 1. Get Top 10 Important Words ---
top_10_words = feature_imp_df.head(10)
print(top_10_words)

        feature  importance
113      return    0.032990
100       excel    0.029485
1         great    0.028984
283       worst    0.028407
130        poor    0.026748
68   disappoint    0.024952
525  your_money    0.018002
167      i_love    0.017955
78     the_best    0.017662
4         don't    0.017654


In [ ]:
near_zero_words_df = feature_imp_df[feature_imp_df['importance'] < 0.00001]


num_eligible = near_zero_words_df.shape[0]

random_10_near_zero = near_zero_words_df.sample(10, random_state=101)

# Reset indices to align them for the table
top_10_words = top_10_words.reset_index(drop=True)
random_10_near_zero = random_10_near_zero.reset_index(drop=True)

table2_df = pd.DataFrame({
    'Important Words': top_10_words['feature'],
    'Unimportant Words': random_10_near_zero['feature']
})

print(table2_df)

  Important Words Unimportant Words
0          return           man_and
1           excel           type_of
2           great             as_my
3           worst            to_use
4            poor          may_have
5      disappoint        the_viewer
6      your_money        the_extras
7          i_love           need_to
8        the_best        reviews_of
9           don't             death


## 2C: Best Random Forest via grid search



This block might take 2-10 minutes. 

If yours runs significantly longer, try this out on Google Colab instead.

In [55]:
randomforest_param_grid = {
    'max_features': [3, 10, 33, 100, 333],
    'max_depth': [16, 32],
    'min_samples_leaf': [1],
    'n_estimators': [100],
    'random_state': [101]
}

In [59]:

# (No need to import sklearn.model; GridSearchCV was already imported earlier)
base_forest = sklearn.ensemble.RandomForestClassifier(
    criterion='gini',
    random_state=101
)

# Use the GridSearchCV symbol already imported earlier
searcher_rf = GridSearchCV(
    estimator=base_forest,
    param_grid=randomforest_param_grid,
    scoring='balanced_accuracy',  # Use balanced_accuracy
    cv=my_split,               # Use our custom train/valid split
    return_train_score=True,
    refit=False,                  # As specified in the instructions
    verbose=2                     # (Optional: Shows progress, helpful!)
)

print("GridSearchCV object for Random Forest created.")

GridSearchCV object for Random Forest created.


### Do the search!

In [60]:
print("Starting Random Forest Grid Search...")
searcher_rf.fit(x_tr_va_NF, y_tr_va_N)
print("... Grid Search complete.")

Starting Random Forest Grid Search...
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.2s
[CV] END max_depth=16, max_features=10, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.5s
[CV] END max_depth=16, max_features=33, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   4.1s
[CV] END max_depth=16, max_features=100, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   9.2s
[CV] END max_depth=16, max_features=333, min_samples_leaf=1, n_estimators=100, random_state=101; total time=  25.5s
[CV] END max_depth=32, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.5s
[CV] END max_depth=32, max_features=10, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   2.7s
[CV] END max_depth=32, max_features=33, min_samples_leaf=1, n_estimators=100, random_state=101; 

### Display search results

In [ ]:
# --- Display search results ---
print("Best parameters found by Random Forest grid search:")
print(searcher_rf.best_params_)
print("\n")

# --- Answer Assignment Questions ---

# 1. What is the value of max_features of your best forest?
best_max_features = searcher_rf.best_params_['max_features']
print(f"Best 'max_features' value: {best_max_features}")

# 2. What is the maximum possible value for max_features?
F = x_tr_NF.shape[1]
print(f"Maximum possible value for 'max_features': {F} (the total number of features)")

print("\nTop 5 results:")
results_rf_df = pd.DataFrame(searcher_rf.cv_results_)
rel_cols_rf = [c for c in results_rf_df.columns if 'param_' in c or 'mean_test_score' in c]
print(results_rf_df[rel_cols_rf].sort_values('mean_test_score', ascending=False).head())

Best parameters found by Random Forest grid search:
{'max_depth': 32, 'max_features': 33, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': 101}


Best 'max_features' value: 33
Maximum possible value for 'max_features': 7729 (the total number of features)

Top 5 results:
  param_max_depth param_max_features param_min_samples_leaf   
7              32                 33                      1  \
6              32                 10                      1   
2              16                 33                      1   
1              16                 10                      1   
3              16                100                      1   

  param_n_estimators param_random_state  mean_test_score  
7                100                101         0.851204  
6                100                101         0.843626  
2                100                101         0.842439  
1                100                101         0.840997  
3                100                101     

Best max_features: 33.

Maximum possible max_features: 7,729 (equal to the total number of features in the dataset).
• It sets how many features are considered at each split.
• Effect:

Lower → more randomization → trees less correlated → lower variance, but higher bias.

Higher → splits use more info → lower bias, but trees become more similar → less variance reduction.

n_estimators tradeoff: More trees → better averaging → lower variance & more stable predictions, but higher compute/memory.
• Generally no for standard Random Forests—the ensemble average converges and test error usually plateaus (or improves slightly). The main cost is computation; not classic overfitting.

### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [65]:
best_forest = sklearn.ensemble.RandomForestClassifier(criterion='gini', random_state=101)
best_forest.set_params(**searcher_rf.best_params_)

RandomForestClassifier(max_depth=32, max_features=33, random_state=101)

In [66]:
best_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=32, max_features=33, random_state=101)

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|

In [ ]:
# List of models and their names for the report
models = {
    "simple Tree": simple_tree,
    "best Tree": best_tree,
    "simple RandomForest": simple_forest,
    "best RandomForest": best_forest
}

# Prepare lists to build the results DataFrame
method_list = []
max_depth_list = []
num_trees_list = []
train_bacc_list = []
valid_bacc_list = []
test_bacc_list = []

for name, model in models.items():
    # Get predictions for all data splits
    y_pred_train = model.predict(x_tr_NF)
    y_pred_valid = model.predict(x_va_NF)
    y_pred_test = model.predict(x_te_NF)

    # Calculate balanced accuracy for each split
    train_bacc = sklearn.metrics.balanced_accuracy_score(y_tr_N, y_pred_train)
    valid_bacc = sklearn.metrics.balanced_accuracy_score(y_va_N, y_pred_valid)
    test_bacc = sklearn.metrics.balanced_accuracy_score(y_te_N, y_pred_test)

    # Store results
    method_list.append(name)
    train_bacc_list.append(round(train_bacc, 3))
    valid_bacc_list.append(round(valid_bacc, 3))
    test_bacc_list.append(round(test_bacc, 3))
    
    # Store model-specific parameters
    max_depth_list.append(model.get_params()['max_depth'])
    if 'RandomForest' in name:
        num_trees_list.append(model.get_params()['n_estimators'])
    else:
        num_trees_list.append(1)

In [69]:
results_summary_df = pd.DataFrame({
    'method': method_list,
    'max_depth': max_depth_list,
    'num_trees': num_trees_list,
    'train_BAcc': train_bacc_list,
    'valid_BAcc': valid_bacc_list,
    'test_BAcc': test_bacc_list
})
results_summary_df

,method,max_depth,num_trees,train_BAcc,valid_BAcc,test_BAcc
0,small Tree,3,1,0.646,0.645,0.646
1,best Tree,32,1,0.877,0.732,0.749
2,simple RandomForest,3,100,0.819,0.797,0.778
3,best RandomForest,32,100,0.964,0.851,0.837


# Conclusion

Hyperparameter Tuning Matters:
Tuning greatly improved performance — the Decision Tree’s test accuracy rose from 0.646 → 0.749, and the Random Forest’s from 0.778 → 0.837. Careful tuning clearly enhances model effectiveness.

Ensemble Models Perform Best:
The Random Forest consistently outperformed single trees; even the untuned version surpassed the tuned Decision Tree. Combining multiple trees helps reduce variance and improves robustness.

Overfitting vs. Generalization:
Deeper models achieved higher training accuracy (e.g., Random Forest: 0.964 train vs. 0.837 test), indicating mild overfitting. Still, they generalized better overall.

I think the tuned Random Forest is the optimal choice, offering the best trade-off between accuracy and generalization.